<a href="https://colab.research.google.com/github/danielturg/Road2_AI_Course_Scripts/blob/main/HW4_Road2_AI_Course_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Preprocessig the images into a folder called 0

In [1]:
import os
import cv2

def crop_resize_grayscale(input_folder, output_folder):
    # Create the output folder
    os.makedirs(output_folder, exist_ok=True)

    # Get a list of image files in the input folder
    image_files = [file for file in os.listdir(input_folder) if file.endswith(".jpg")]

    # Process each image file
    for image_file in image_files:
        # Load the image
        image_path = os.path.join(input_folder, image_file)
        image = cv2.imread(image_path)

        # Load the label
        label_path = os.path.join(input_folder, image_file.replace('.jpg', '.txt'))
        label_values = open(label_path).readline().split()

        # Extract bounding box values from the label
        contains_drone = int(label_values[0])
        x_center, y_center, width, height = map(float, label_values[1:])

        # Compute bounding box coordinates
        xmin = int(max(0, (x_center - 0.5 * width) * image.shape[1]))
        ymin = int(max(0, (y_center - 0.5 * height) * image.shape[0]))
        xmax = int(min(image.shape[1], (x_center + 0.5 * width) * image.shape[1]))
        ymax = int(min(image.shape[0], (y_center + 0.5 * height) * image.shape[0]))

        # Crop and resize the image
        cropped_image = image[ymin:ymax, xmin:xmax]
        resized_image = cv2.resize(cropped_image, (28, 28))

        # Convert the image to grayscale
        grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

        # Save the processed image
        output_image_path = os.path.join(output_folder, image_file)
        cv2.imwrite(output_image_path, grayscale_image)

        # Save the corresponding label
        output_label_path = os.path.join(output_folder, image_file.replace('.jpg', '.txt'))
        with open(output_label_path, 'w') as label_file:
            label_file.write(str(contains_drone))

# Example usage
input_folder = "/content/drive/MyDrive/AI_Course/drone_data"
output_folder = "/content/drive/MyDrive/AI_Course/Processed_Dataset/0"
crop_resize_grayscale(input_folder, output_folder)

print("Image processing completed.")


Image processing completed.


Preprocessing the images with fake bounding boxes into a folder called "1"

In [2]:
import os
import cv2
import numpy as np

def generate_fake_labels(image_shape):
    # Generate random bounding box values
    x_center = np.random.uniform(0.2, 0.8)
    y_center = np.random.uniform(0.2, 0.8)
    width = np.random.uniform(0.2, 0.4)
    height = np.random.uniform(0.2, 0.4)

    # Compute bounding box coordinates
    xmin = int(max(0, (x_center - 0.5 * width) * image_shape[1]))
    ymin = int(max(0, (y_center - 0.5 * height) * image_shape[0]))
    xmax = int(min(image_shape[1], (x_center + 0.5 * width) * image_shape[1]))
    ymax = int(min(image_shape[0], (y_center + 0.5 * height) * image_shape[0]))

    return xmin, ymin, xmax, ymax

def crop_resize_grayscale_fake(input_folder, output_folder):
    # Create the output folder
    os.makedirs(output_folder, exist_ok=True)

    # Get a list of image files in the input folder
    image_files = [file for file in os.listdir(input_folder) if file.endswith(".jpg")]

    # Process each image file
    for image_file in image_files:
        # Load the image
        image_path = os.path.join(input_folder, image_file)
        image = cv2.imread(image_path)

        # Generate fake label values
        xmin, ymin, xmax, ymax = generate_fake_labels(image.shape)

        # Crop and resize the image
        cropped_image = image[ymin:ymax, xmin:xmax]
        resized_image = cv2.resize(cropped_image, (28, 28))

        # Convert the image to grayscale
        grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

        # Save the processed image with "_fake" suffix
        output_image_path = os.path.join(output_folder, image_file.replace('.jpg', '_fake.jpg'))
        cv2.imwrite(output_image_path, grayscale_image)

        # Save the corresponding fake label with "_fake" suffix
        output_label_path = os.path.join(output_folder, image_file.replace('.jpg', '_fake.txt'))
        with open(output_label_path, 'w') as label_file:
            label_file.write("1")  # Assuming the label for fake images is always 1

# Example usage
input_folder = "/content/drive/MyDrive/AI_Course/drone_data"
output_folder = "/content/drive/MyDrive/AI_Course/Processed_Dataset/1"
crop_resize_grayscale_fake(input_folder, output_folder)

print("Fake image processing completed.")


Fake image processing completed.


splitting the dataset into train and test

In [3]:
import os
import shutil
import random

def split_data(input_folder, output_folder, train_ratio=0.8):
    # Create output folders
    train_folder = os.path.join(output_folder, 'train')
    test_folder = os.path.join(output_folder, 'test')
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)

    # Get a list of image files in the input folder
    image_files_0 = [file for file in os.listdir(os.path.join(input_folder, '0')) if file.endswith(".jpg")]
    image_files_1 = [file for file in os.listdir(os.path.join(input_folder, '1')) if file.endswith("_fake.jpg")]

    # Shuffle the data
    random.shuffle(image_files_0)
    random.shuffle(image_files_1)

    # Determine the number of files for training and testing
    num_train_0 = int(len(image_files_0) * train_ratio)
    num_test_0 = len(image_files_0) - num_train_0
    num_train_1 = int(len(image_files_1) * train_ratio)
    num_test_1 = len(image_files_1) - num_train_1

    # Copy 80% of the data from "0" and "1" to the "train" folder
    for image_file in image_files_0[:num_train_0]:
        shutil.copy(os.path.join(input_folder, '0', image_file), os.path.join(train_folder, image_file))
        shutil.copy(os.path.join(input_folder, '0', image_file.replace('.jpg', '.txt')),
                    os.path.join(train_folder, image_file.replace('.jpg', '.txt')))

    for image_file in image_files_1[:num_train_1]:
        shutil.copy(os.path.join(input_folder, '1', image_file), os.path.join(train_folder, image_file))
        shutil.copy(os.path.join(input_folder, '1', image_file.replace('_fake.jpg', '_fake.txt')),
                    os.path.join(train_folder, image_file.replace('_fake.jpg', '_fake.txt')))

    # Copy 20% of the data from "0" and "1" to the "test" folder
    for image_file in image_files_0[-num_test_0:]:
        shutil.copy(os.path.join(input_folder, '0', image_file), os.path.join(test_folder, image_file))
        shutil.copy(os.path.join(input_folder, '0', image_file.replace('.jpg', '.txt')),
                    os.path.join(test_folder, image_file.replace('.jpg', '.txt')))

    for image_file in image_files_1[-num_test_1:]:
        shutil.copy(os.path.join(input_folder, '1', image_file), os.path.join(test_folder, image_file))
        shutil.copy(os.path.join(input_folder, '1', image_file.replace('_fake.jpg', '_fake.txt')),
                    os.path.join(test_folder, image_file.replace('_fake.jpg', '_fake.txt')))

# Example usage
input_folder = "/content/drive/MyDrive/AI_Course/Processed_Dataset"
output_folder = "/content/drive/MyDrive/AI_Course/Processed_Dataset"
split_data(input_folder, output_folder)

print("Data splitting completed.")


Data splitting completed.


building and training the model- loading data and libraries

In [1]:
import os
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

def load_data(folder_path):
    images = []
    labels = []

    image_files = [file for file in os.listdir(folder_path) if file.endswith(".jpg")]
    for image_file in image_files:
        # Load and preprocess the image
        image_path = os.path.join(folder_path, image_file)
        image = load_img(image_path, color_mode='grayscale', target_size=(28, 28))
        image_array = img_to_array(image) / 255.0  # Normalize pixel values to be between 0 and 1
        images.append(image_array)

        # Load and preprocess the label
        label_path = os.path.join(folder_path, image_file.replace('.jpg', '.txt'))
        label = int(open(label_path).readline().strip())
        labels.append(label)

    return np.array(images), np.array(labels)

# Load the data
train_folder = "/content/drive/MyDrive/AI_Course/Processed_Dataset/train"
test_folder = "/content/drive/MyDrive/AI_Course/Processed_Dataset/test"

X_train, y_train = load_data(train_folder)
X_val, y_val = load_data(test_folder)  # Use "test" folder as validation set




Epoch 1/10


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


ValueError: ignored

In [4]:
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy

building and training the network

In [5]:
# Create the neural network model
model = Sequential([
    Flatten(input_shape=(28, 28, 1)),
    Dense(1, activation='sigmoid')  # Change activation to 'sigmoid' for binary classification
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=BinaryCrossentropy(),
    metrics=[BinaryAccuracy()]
)

# Train the model
epochs = 10
batch_size = 32

for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=1,
        batch_size=batch_size,
        verbose=1
    )

    # Evaluate the model on the validation set
    val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
    print(f"Validation Accuracy: {val_accuracy}")

print("Training completed.")


Epoch 1/10
204/204 [==============================] - 6s 5ms/step - loss: 0.5237 - binary_accuracy: 0.7786 - val_loss: 0.4352 - val_binary_accuracy: 0.8563
Validation Accuracy: 0.8562653660774231

Epoch 2/10
204/204 [==============================] - 1s 6ms/step - loss: 0.4403 - binary_accuracy: 0.8303 - val_loss: 0.4827 - val_binary_accuracy: 0.6708
Validation Accuracy: 0.6707616448402405

Epoch 3/10
204/204 [==============================] - 1s 5ms/step - loss: 0.4294 - binary_accuracy: 0.8332 - val_loss: 0.3640 - val_binary_accuracy: 0.8692
Validation Accuracy: 0.8691646456718445

Epoch 4/10
204/204 [==============================] - 1s 5ms/step - loss: 0.4119 - binary_accuracy: 0.8401 - val_loss: 0.5208 - val_binary_accuracy: 0.5958
Validation Accuracy: 0.5958231091499329

Epoch 5/10
204/204 [==============================] - 1s 5ms/step - loss: 0.3985 - binary_accuracy: 0.8509 - val_loss: 0.4746 - val_binary_accuracy: 0.7015
Validation Accuracy: 0.7014741897583008

Epoch 6/10
204

predicting:

In [7]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Path to the image you want to predict
path_to_image = "/content/drive/MyDrive/AI_Course/Detect-a-Drone-in-the-Skysss.jpg"

# Load and preprocess the image
img = load_img(path_to_image, color_mode='grayscale', target_size=(28, 28))
img_array = img_to_array(img) / 255.0  # Normalize pixel values to be between 0 and 1
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Make a prediction using the trained model
prediction = model.predict(img_array)

# Convert the prediction to a human-readable label
class_label = "Drone" if prediction[0][0] < 0.5 else "No Drone"

print(f"The model predicts: {class_label}")


1/1 [==============================] - 0s 41ms/step
The model predicts: No Drone
